# Rates Simulations

Построение симуляций для моделирования поведения курсов валют (пары USD/RUB и EUR/RUB)

В качестве модели будем использовать логарифмическую модель

In [21]:
import pandas as pd
import numpy as np

from data import DATA_PATH

### Чтение данных

In [44]:
all_data = pd.read_csv(DATA_PATH / 'all_data.csv', index_col='date')
all_data.head()

,su26230_days_before_coupon,su26224_days_before_coupon,su26222_days_before_coupon,su26221_days_before_coupon,su26218_days_before_coupon,ecb_rate,aluminum,brent,cbr_key_rate,eur_rub,...,sber,vtbr,year_1,year_3,year_5,year_10,year_15,year_20,pca_cbd,sofr
date,,,,,,,,,,,,,,,,,,,,,
2020-01-10,89,145,103,89,82,0.25,1806.0,65.02,6.25,68.0555,...,258.19,0.04760,5.48,5.69,5.90,6.26,6.48,6.61,-5.601225,1.55
2020-01-11,89,145,103,89,82,0.25,1806.0,65.02,6.25,68.0555,...,258.19,0.04760,5.48,5.69,5.90,6.26,6.48,6.61,-5.601225,1.55
2020-01-12,89,145,103,89,82,0.25,1806.0,65.02,6.25,68.0555,...,258.19,0.04760,5.48,5.69,5.90,6.26,6.48,6.61,-5.601225,1.55
2020-01-13,86,142,100,86,79,0.25,1798.0,64.23,6.25,68.0450,...,262.40,0.04806,5.45,5.64,5.84,6.18,6.39,6.52,-5.760549,1.54
2020-01-14,85,141,99,85,78,0.25,1809.0,64.42,6.25,67.8162,...,259.05,0.04726,5.46,5.68,5.89,6.26,6.47,6.60,-5.626252,1.55


In [23]:
usd_rub = all_data['usd_rub']

In [24]:
usd_rub.describe()

count    1450.000000
mean       74.659637
std         9.839833
min        51.158000
25%        70.543975
50%        73.786400
75%        77.732500
max       103.161800
Name: usd_rub, dtype: float64

In [25]:
eur_rub = all_data['eur_rub']

In [26]:
eur_rub.describe()

count    1450.000000
mean       83.350778
std        12.260458
min        52.737900
25%        77.796500
50%        86.345700
75%        90.202300
max       113.261300
Name: eur_rub, dtype: float64

### Формальная постановка задачи

**Логарифмическая модель:**

$
\frac{d X_t}{X_t} = (r_t^f - r_t^d)d t + \sigma_1 d W_t
$

- $r^d$ - мгновенная процентная ставка валюты Рубль

- $r^f$ - мгновенная процентная ставка валюты Доллар/Евро

Разностная схема: $X_{t+1} = X_t + X_t(r_t^f - r_t^d)d t + X_t \sigma_1(W_{t+1} - W_t)$

In [18]:
def log_sim(x_0, r_f, r_d, sigma, n_days, n_sim, deltas_W=None):
    """
    x_0 - float
    r_f - np.array[n_sim x (n_days + 1)]
    r_d - np.array[n_sim x (n_days + 1)]
    sigma - float
    n_sim - int

    result - np.array[n_sim x (n_days + 1)]
    """
    result = np.array([x_0] * n_sim).reshape(-1, 1)
    for i in range(1, n_days + 1):
        x_prev = result[:, -1].reshape(-1, 1)
        delta_t = 1

        if deltas_W is None:
            delta_W = np.random.normal(loc=0, scale=np.sqrt(delta_t), size=(n_sim, 1))
        else:
            delta_W = deltas_W[:, i-1].reshape(-1, 1)

        r_d_i = r_d[:,i-1].reshape(-1, 1)
        r_f_i = r_f[:,i-1].reshape(-1, 1)

        x_t_i = x_prev + x_prev *(r_f_i - r_d_i) * delta_t + sigma * x_prev * delta_W
        result = np.hstack([result, x_t_i])
    return result

### Подберем параметры для модели

Для того, чтобы оценить сигму для лог модели, мы воспользовались схемой Эйлера-Муроямы, выразив шум модели, посчитали шум на исторических данных и взяли его стандартное отклонение

In [34]:
def calc_opt_sigma(
    fx_data: pd.Series,
    domestic_rates: pd.Series,
    foreign_rates: pd.Series,
) -> float:
    domestic_prev = domestic_rates.values[:-1]
    foreign_prev = foreign_rates.values[:-1]
    fx_prev = fx_data.values[:-1]
    fx_curr = fx_data.values[1:]
    noise = fx_curr - fx_prev * (1 - foreign_prev - domestic_prev)
    noise_fx = noise / fx_prev
    return np.std(noise_fx)

In [37]:
calc_opt_sigma(usd_rub, all_data['cbr_key_rate'], all_data['ecb_rate'])

4.351965526837073

In [45]:
calc_opt_sigma(eur_rub, all_data['cbr_key_rate'], all_data['sofr'])

4.673321811786814

Стоит использовать разложение Холецкого для построения скоррелированных симуляций (реализацию см. в коде решения)